# Steepest Descent Example

In this example, the 1CTF protein sequence will be used to exemplify the ProtoSyn Steepest Descent Driver. The protein dihedrals will be approximated to the native structure, causing some overlaps in the process. Using a Steepest Descent algorithm, ProtoSyn is then able to relax the structure and the conflicts.

In [1]:
using ProtoSyn;
using LinearAlgebra;
using Printf;

## 1. Load a state

The first step is to load a starting state from a structural file. ProtoSyn enables loading of coordinates from several standardized file extensions, such as PDB. In this example, the `1ctf.pdb` file was produced using an auxiliar tool that places the amicoacid sequence in a straight configuration, with all dihedrals as 180º. This will be the simulation starting point. 

In [2]:
state = Common.load_from_pdb("data/1ctf.pdb");
state.energy = Forcefield.Amber.Energy();

## 2. (Optional) Apply dihedrals from a second structural file

For this particular example, the native structure of the 1CTF protein will be used as a template to apply the dihedral angles to the loaded `State`, approximating the 2 structures.

In [3]:
mc_topology = Aux.read_JSON("data/1ctf_mc_top.json");
dihedrals, residues = Common.load_topology(mc_topology);
bb_dihedrals = filter(x -> x.dtype < Common.omega, dihedrals);
Common.apply_dihedrals_from_file(state, bb_dihedrals, "data/1ctf_native.pdb");

## 3. (Optional) Fix the damn Prolines

Let's be honest here: nobody likes the Prolines. They are weird and ugly. This function will most likely be fixed in later versions of the ProtoSyn library.

In [4]:
Common.fix_proline(state, dihedrals);

## 4. Define the Evaluator

In any Steepest Descent algorithm, the direction of the system is defined by the evolution of the energy. The **evaluator** here is responsible to calculate the energy. In this example, the Amber forcefield will be employed. For this, ProtoSyn needs to have access to information regarding the bonds, angles, dihedrals and non-bonded interactions of the molecule, avaliable by loading a topology file.

In [5]:
topology = Forcefield.Amber.load_from_json("data/1ctf_amber_top.json");
function my_evaluator!(st::Common.State, do_forces::Bool)
    energy = Forcefield.Amber.evaluate!(topology, st, cut_off=1.0, do_forces=do_forces)
    return energy
end

my_evaluator! (generic function with 1 method)

## 5. Define the Driver

At this point, the system is ready to run. A **Driver** is a predefined way of incorporating the **Evaluator** in order to take the system from one stage to the next, for a certain amount of steps or until the convergence criteria are met. In this example, the SteepestDescentDriver will be employed, that evaluates the energy **and forces** of the system, relaxing the bonds, angles, dihedrals and non-bonded interactions of the molecule.

In [6]:
sd_driver = Drivers.SteepestDescent.SteepestDescentDriver(my_evaluator!, n_steps = 1000, f_tol = 0.02)

SteepestDescentDriver(evaluator=my_evaluator!, n_steps=1000, f_tol=0.02)

## 6. (Optional) Define the Callbacks

Altough the system is ready to run, no output is being printed. In order to gain access to the inner working of the **Driver** callbacks need to be defined.
The **CallbackObject** will print the current structure to a .pdb file and the system status to the terminal every 25 steps.

In [7]:
file_xyz = open("out/trajectory.pdb", "w")
my_callback = @Common.callback 100 function callback_status(step::Int64, st::Common.State, dr::Drivers.SteepestDescent.SteepestDescentDriver, args...)
    write(stdout, @sprintf "(SD) Step: %4d | Energy: %4.4e | Max Force: %4.4e | Gamma: %4.4e\n" step state.energy.eTotal args[1] args[2])
    Print.as_pdb(file_xyz, st, step=step)
end

CallbackObject(freq=100, callback=callback_status)

## 7. Make science!

The set up of the system is complete, run the **Driver** and analyse the output!

In [8]:
Drivers.SteepestDescent.run!(state, sd_driver, my_callback)
close(file_xyz)

(SD) Step:    0 | Energy: 1.4112e+11 | Max Force: 1.4371e+13 | Gamma: 1.0000e+00
(SD) Step:  100 | Energy: 1.1167e+04 | Max Force: 1.6587e+04 | Gamma: 1.3621e-02
(SD) Step:  200 | Energy: 6.7035e+03 | Max Force: 1.0554e+03 | Gamma: 3.8597e-10
Gamma below machine precision! Exiting ...

